In [ ]:
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
import os, shutil
import io, requests
import zipfile
import joblib

data_path = 'datas'
os.makedirs(data_path, exist_ok=True)

# Trafic moyen journalier

In [ ]:
# tmja = gpd.read_file('C:/Users/demo/Desktop/Lattitude/datas/tmja2018-shp/TMJA2018.shp')
file_name = 'TMJA2018'
ext = '.shp'
url = 'https://static.data.gouv.fr/resources/trafic-moyen-journalier-annuel-sur-le-reseau-routier-national/20200601-112539/tmja2018-shp.zip'
temp_path = 'temp_unzip'

try: 
    tmja = gpd.read_feather(os.path.join(data_path,file_name + '.feather'))
except:
    # From url
    zip_file = requests.get(url).content
    os.makedirs(temp_path, exist_ok=True)
    with zipfile.ZipFile(io.BytesIO(zip_file)) as archive:
        archive.extractall(temp_path)
    tmja = gpd.read_file(os.path.join(temp_path,file_name + ext))
    shutil.rmtree(temp_path)

    # From local file
    # tmja = gpd.read_file(os.path.join(data_path,file_name + ext))

    # avoiding multi dowloads for GES emmision lowering
    tmja.to_feather(os.path.join(data_path,file_name + '.feather'))

In [ ]:
tmja.info()

In [ ]:
tmja.isna().any()

In [ ]:
tmja.geometry[:1]

In [ ]:
tmja.tmja.plot(kind='hist',column='tmja', cmap='viridis', figsize=(12, 8), legend=True)

In [ ]:
tmja.tmja.mean()

# Style

In [ ]:
from branca.colormap import linear, LinearColormap

min = tmja['tmja'].min()
max = tmja['tmja'].max()
print(f'trafic | min: {min} | max: {max}')

# Define the color map
colors = ['#00ff00', '#ffff00', '#ff0000']
cmap = LinearColormap(colors=colors, vmin=min, vmax=max)
display(cmap)

# Create a color ramp
colormap = linear.YlGnBu_09.scale(
    tmja['tmja'].min(),
    tmja['tmja'].max())

display(colormap)
colormap(23320)

In [ ]:
# Color by feature
def style_function_color(feature):
    color = 'red' if feature['properties']['tmja'] > 23320 else 'green'
    return {'fillColor': color, 'color': 'black'}


# Define the style function Gradien color
def style_function_gradien(feature):
    color = cmap(feature['properties']['tmja'])
    return {'weight': 3,
            'color': color,
            'opacity': .9,
            # 'fillColor': cmap,
            # 'fillOpacity': 1,
             }


In [ ]:
trafic = folium.GeoJson(tmja, style_function=style_function_gradien, overlay=True, name='Trafic')

In [ ]:
marker_path = 'datas/markers'
os.makedirs(marker_path, exist_ok=True)

joblib.dump(trafic, os.path.join(marker_path,'trafic_journalier.mrkr'))

# Carte

In [ ]:
# Create a folium map object
map = folium.Map(location= [48.7, 2.3],
                    width='100%', height='100%', left='0%', top='0%', position='relative', 
                    tiles= 'OpenStreetMap',     #OpenStreetMap', Stamen
                    attr=None,    #  None   Terrain, Toner, and Watercolor
                    min_zoom=0, max_zoom=18, zoom_start=9, 
                    min_lat=- 90, max_lat=90, min_lon=- 180, max_lon=180, max_bounds=False, 
                    crs='EPSG3857', control_scale=False, prefer_canvas=False, 
                    no_touch=False, disable_3d=False, png_enabled=False, zoom_control=True)



# # Add the GeoJSON object to the map
trafic.add_to(map)

# # Add a layer control to the map
folium.LayerControl().add_to(map)

# # Display the map in the notebook
map


In [ ]:
maps_path = 'datas/maps'
os.makedirs(maps_path, exist_ok=True)

map.save(os.path.join(maps_path,'carte_traffic_journalier.html'))